# Ugesco : enrich and desambiguate a sample of the Samnang-Hans' JSON

In [1]:
from ugesco import *

D:\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
D:\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
D:\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
D:\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
D:\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
D:\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, fa

In [2]:
#test sur un Json contenant 5 éléments pris au hasard
import json
import random
with open(r"C:\Users\ettor\Documents\GitHub\UGESCO\data\jsondata_ugesco.json", "r", encoding="utf8") as file:
    json = json.load(file)
sample = random.sample(json, k=5)


In [3]:
data = pd.DataFrame([flatten(line) for line in sample])

In [4]:
data.head()

,beeldid,imageclassification_class1,imageclassification_class2,imageclassification_class3,imageclassification_class4,imageclassification_class5,imageclassification_prob1,imageclassification_prob2,imageclassification_prob3,imageclassification_prob4,imageclassification_prob5,spatial_key_0,spatial_key_1,spatial_key_2,spatial_value_0,spatial_value_1,spatial_value_2,temporal
0,00038225,plaza,auditorium,television_studio,courtyard,stage/indoor,0.193922,0.052447,0.035257,0.035217,0.029309,LOC_BUILDING,NaN,NaN,"Lycée Commun , d' Ixelles",NaN,NaN,None
1,00005881,street,embassy,plaza,office_building,apartment_building/outdoor,0.377983,0.327560,0.119684,0.036492,0.028549,LOC_PLACE,NaN,NaN,Bruges,NaN,NaN,None
2,00029746,apartment_building/outdoor,building_facade,street,aqueduct,office_building,0.624949,0.072448,0.072407,0.044429,0.029015,LOC_PLACE,LOC_BUILDING,LOC_PLACE,Verviers,Le Palais de Justice,Verviers,None
3,00029795,palace,courthouse,plaza,library/outdoor,church/outdoor,0.333751,0.116114,0.085986,0.084669,0.056719,LOC_BUILDING,LOC_PLACE,NaN,l' hôtel de ville,Verviers,NaN,None
4,00010703,bazaar/outdoor,market/outdoor,street,flea_market/indoor,slum,0.845973,0.059354,0.023403,0.022964,0.013281,LOC_PLACE,NaN,NaN,Malines,NaN,NaN,None


In [5]:
# exclude image classifications and temporal values for the moment
data = data[data.columns.drop(list(data.filter(regex='imageclassification|temporal')))]

In [6]:
# reshape the dataframe verticaly
data = data.set_index('beeldid')
data.columns = data.columns.str.split('_', expand=True)

In [7]:
data = data.stack().reset_index(level=1, drop=True).reset_index()

In [8]:
data.head()

beeldid       spatial                           
                      key                      value
0  00038225  LOC_BUILDING  Lycée Commun , d' Ixelles
1  00005881     LOC_PLACE                     Bruges
2  00029746     LOC_PLACE                   Verviers
3  00029746  LOC_BUILDING       Le Palais de Justice
4  00029746     LOC_PLACE                   Verviers

In [9]:
data.columns = ['_'.join(col) for col in data.columns]

In [10]:
data.rename(columns={'beeldid_': 'beeldid'}, inplace=True)

In [11]:
data.head()

,beeldid,spatial_key,spatial_value
0,00038225,LOC_BUILDING,"Lycée Commun , d' Ixelles"
1,00005881,LOC_PLACE,Bruges
2,00029746,LOC_PLACE,Verviers
3,00029746,LOC_BUILDING,Le Palais de Justice
4,00029746,LOC_PLACE,Verviers


In [12]:
#join/merge with phototheque_pallas to get the locations. 
#The csv is gzipped to get around the 100MB file weigth limitation of Github
phototeque = pd.read_csv("data/phototheque_pallas.csv.gz", compression="gzip", encoding="utf8", dtype={'beeldid': str})

data = pd.merge(data, phototeque, how='left', on=['beeldid'], suffixes=['', '_x'])

In [13]:
# Pictures that contains more than a loc in the thesaurus descriptors

#data[data['loc_qid'].str.contains(",", na = False)]

In [14]:
data.head()

,beeldid,spatial_key,spatial_value,lien_phototheque,LEGEND,BPALLASF,trfwnumm,keyword,country_qid,country_name,loc_qid,loc_name
0,00038225,LOC_BUILDING,"Lycée Commun , d' Ixelles",http://pallas.cegesoma.be/pls/opac/opac.search...,"Les élèves du Lycée Commun, d'Ixelles se repos...","Danse, cirque, variétés et théâtre : [1936-193...",4349.0,danse--belgique,Q31,Belgium,NaN,NaN
1,00005881,LOC_PLACE,Bruges,http://pallas.cegesoma.be/pls/opac/opac.search...,Ravitaillement à Bruges. [foto Huis Van Eyck],"Vie quotidienne à Bruges, 1940-1945. - 8 photos",241.0,vie pratique--brugge (belgique),Q31,Belgium,Q12994,Bruges
2,00029746,LOC_PLACE,Verviers,http://pallas.cegesoma.be/pls/opac/opac.search...,Jour de la libération de Verviers. Le Palais d...,"Libération de Verviers par les alliés, 9/1944....",1764.0,verviers (belgique)--histoire--1944-1945 (libe...,Q31,Belgium,Q202954,Verviers
3,00029746,LOC_BUILDING,Le Palais de Justice,http://pallas.cegesoma.be/pls/opac/opac.search...,Jour de la libération de Verviers. Le Palais d...,"Libération de Verviers par les alliés, 9/1944....",1764.0,verviers (belgique)--histoire--1944-1945 (libe...,Q31,Belgium,Q202954,Verviers
4,00029746,LOC_PLACE,Verviers,http://pallas.cegesoma.be/pls/opac/opac.search...,Jour de la libération de Verviers. Le Palais d...,"Libération de Verviers par les alliés, 9/1944....",1764.0,verviers (belgique)--histoire--1944-1945 (libe...,Q31,Belgium,Q202954,Verviers


In [15]:
# Merge with NER_classes (spatial_keys matched with wikidata)
ner_classes = pd.read_csv("data/ner_classes.csv")

data = pd.merge(data, ner_classes, how='left', left_on = ['spatial_key'], right_on = ['ner_class'])

In [16]:
data.head()

,beeldid,spatial_key,spatial_value,lien_phototheque,LEGEND,BPALLASF,trfwnumm,keyword,country_qid,country_name,loc_qid,loc_name,ner_class,wiki_qid,wiki_class
0,00038225,LOC_BUILDING,"Lycée Commun , d' Ixelles",http://pallas.cegesoma.be/pls/opac/opac.search...,"Les élèves du Lycée Commun, d'Ixelles se repos...","Danse, cirque, variétés et théâtre : [1936-193...",4349.0,danse--belgique,Q31,Belgium,NaN,NaN,LOC_BUILDING,Q41176,building
1,00005881,LOC_PLACE,Bruges,http://pallas.cegesoma.be/pls/opac/opac.search...,Ravitaillement à Bruges. [foto Huis Van Eyck],"Vie quotidienne à Bruges, 1940-1945. - 8 photos",241.0,vie pratique--brugge (belgique),Q31,Belgium,Q12994,Bruges,LOC_PLACE,Q56061,administrative territorial entity
2,00029746,LOC_PLACE,Verviers,http://pallas.cegesoma.be/pls/opac/opac.search...,Jour de la libération de Verviers. Le Palais d...,"Libération de Verviers par les alliés, 9/1944....",1764.0,verviers (belgique)--histoire--1944-1945 (libe...,Q31,Belgium,Q202954,Verviers,LOC_PLACE,Q56061,administrative territorial entity
3,00029746,LOC_BUILDING,Le Palais de Justice,http://pallas.cegesoma.be/pls/opac/opac.search...,Jour de la libération de Verviers. Le Palais d...,"Libération de Verviers par les alliés, 9/1944....",1764.0,verviers (belgique)--histoire--1944-1945 (libe...,Q31,Belgium,Q202954,Verviers,LOC_BUILDING,Q41176,building
4,00029746,LOC_PLACE,Verviers,http://pallas.cegesoma.be/pls/opac/opac.search...,Jour de la libération de Verviers. Le Palais d...,"Libération de Verviers par les alliés, 9/1944....",1764.0,verviers (belgique)--histoire--1944-1945 (libe...,Q31,Belgium,Q202954,Verviers,LOC_PLACE,Q56061,administrative territorial entity


In [17]:
# rename the new columns merged
data.rename(columns={'wiki_qid': 'ner_class_qid', 'wiki_class':'ner_class_name'}, inplace=True)

In [18]:
data.head()

,beeldid,spatial_key,spatial_value,lien_phototheque,LEGEND,BPALLASF,trfwnumm,keyword,country_qid,country_name,loc_qid,loc_name,ner_class,ner_class_qid,ner_class_name
0,00038225,LOC_BUILDING,"Lycée Commun , d' Ixelles",http://pallas.cegesoma.be/pls/opac/opac.search...,"Les élèves du Lycée Commun, d'Ixelles se repos...","Danse, cirque, variétés et théâtre : [1936-193...",4349.0,danse--belgique,Q31,Belgium,NaN,NaN,LOC_BUILDING,Q41176,building
1,00005881,LOC_PLACE,Bruges,http://pallas.cegesoma.be/pls/opac/opac.search...,Ravitaillement à Bruges. [foto Huis Van Eyck],"Vie quotidienne à Bruges, 1940-1945. - 8 photos",241.0,vie pratique--brugge (belgique),Q31,Belgium,Q12994,Bruges,LOC_PLACE,Q56061,administrative territorial entity
2,00029746,LOC_PLACE,Verviers,http://pallas.cegesoma.be/pls/opac/opac.search...,Jour de la libération de Verviers. Le Palais d...,"Libération de Verviers par les alliés, 9/1944....",1764.0,verviers (belgique)--histoire--1944-1945 (libe...,Q31,Belgium,Q202954,Verviers,LOC_PLACE,Q56061,administrative territorial entity
3,00029746,LOC_BUILDING,Le Palais de Justice,http://pallas.cegesoma.be/pls/opac/opac.search...,Jour de la libération de Verviers. Le Palais d...,"Libération de Verviers par les alliés, 9/1944....",1764.0,verviers (belgique)--histoire--1944-1945 (libe...,Q31,Belgium,Q202954,Verviers,LOC_BUILDING,Q41176,building
4,00029746,LOC_PLACE,Verviers,http://pallas.cegesoma.be/pls/opac/opac.search...,Jour de la libération de Verviers. Le Palais d...,"Libération de Verviers par les alliés, 9/1944....",1764.0,verviers (belgique)--histoire--1944-1945 (libe...,Q31,Belgium,Q202954,Verviers,LOC_PLACE,Q56061,administrative territorial entity


In [19]:
# remove articles in spatial_value
data['spatial_value'] = data['spatial_value'].str.replace(pat=r"^\s?(le|la|l\s+'|l'|les)\s+", repl="", n=1, case=False)

In [20]:
#data.to_csv(r"C:\Users\ettor\Desktop\data_ugesco.csv", header=True, index=False, encoding="utf-8")

In [21]:
data.head()

,beeldid,spatial_key,spatial_value,lien_phototheque,LEGEND,BPALLASF,trfwnumm,keyword,country_qid,country_name,loc_qid,loc_name,ner_class,ner_class_qid,ner_class_name
0,00038225,LOC_BUILDING,"Lycée Commun , d' Ixelles",http://pallas.cegesoma.be/pls/opac/opac.search...,"Les élèves du Lycée Commun, d'Ixelles se repos...","Danse, cirque, variétés et théâtre : [1936-193...",4349.0,danse--belgique,Q31,Belgium,NaN,NaN,LOC_BUILDING,Q41176,building
1,00005881,LOC_PLACE,Bruges,http://pallas.cegesoma.be/pls/opac/opac.search...,Ravitaillement à Bruges. [foto Huis Van Eyck],"Vie quotidienne à Bruges, 1940-1945. - 8 photos",241.0,vie pratique--brugge (belgique),Q31,Belgium,Q12994,Bruges,LOC_PLACE,Q56061,administrative territorial entity
2,00029746,LOC_PLACE,Verviers,http://pallas.cegesoma.be/pls/opac/opac.search...,Jour de la libération de Verviers. Le Palais d...,"Libération de Verviers par les alliés, 9/1944....",1764.0,verviers (belgique)--histoire--1944-1945 (libe...,Q31,Belgium,Q202954,Verviers,LOC_PLACE,Q56061,administrative territorial entity
3,00029746,LOC_BUILDING,Palais de Justice,http://pallas.cegesoma.be/pls/opac/opac.search...,Jour de la libération de Verviers. Le Palais d...,"Libération de Verviers par les alliés, 9/1944....",1764.0,verviers (belgique)--histoire--1944-1945 (libe...,Q31,Belgium,Q202954,Verviers,LOC_BUILDING,Q41176,building
4,00029746,LOC_PLACE,Verviers,http://pallas.cegesoma.be/pls/opac/opac.search...,Jour de la libération de Verviers. Le Palais d...,"Libération de Verviers par les alliés, 9/1944....",1764.0,verviers (belgique)--histoire--1944-1945 (libe...,Q31,Belgium,Q202954,Verviers,LOC_PLACE,Q56061,administrative territorial entity


In [22]:
gb = data.groupby(['beeldid'])

In [28]:
gb.head()

,beeldid,spatial_key,spatial_value,lien_phototheque,LEGEND,BPALLASF,trfwnumm,keyword,country_qid,country_name,loc_qid,loc_name,ner_class,ner_class_qid,ner_class_name
0,00038225,LOC_BUILDING,"Lycée Commun , d' Ixelles",http://pallas.cegesoma.be/pls/opac/opac.search...,"Les élèves du Lycée Commun, d'Ixelles se repos...","Danse, cirque, variétés et théâtre : [1936-193...",4349.0,danse--belgique,Q31,Belgium,NaN,NaN,LOC_BUILDING,Q41176,building
1,00005881,LOC_PLACE,Bruges,http://pallas.cegesoma.be/pls/opac/opac.search...,Ravitaillement à Bruges. [foto Huis Van Eyck],"Vie quotidienne à Bruges, 1940-1945. - 8 photos",241.0,vie pratique--brugge (belgique),Q31,Belgium,Q12994,Bruges,LOC_PLACE,Q56061,administrative territorial entity
2,00029746,LOC_PLACE,Verviers,http://pallas.cegesoma.be/pls/opac/opac.search...,Jour de la libération de Verviers. Le Palais d...,"Libération de Verviers par les alliés, 9/1944....",1764.0,verviers (belgique)--histoire--1944-1945 (libe...,Q31,Belgium,Q202954,Verviers,LOC_PLACE,Q56061,administrative territorial entity
3,00029746,LOC_BUILDING,Palais de Justice,http://pallas.cegesoma.be/pls/opac/opac.search...,Jour de la libération de Verviers. Le Palais d...,"Libération de Verviers par les alliés, 9/1944....",1764.0,verviers (belgique)--histoire--1944-1945 (libe...,Q31,Belgium,Q202954,Verviers,LOC_BUILDING,Q41176,building
4,00029746,LOC_PLACE,Verviers,http://pallas.cegesoma.be/pls/opac/opac.search...,Jour de la libération de Verviers. Le Palais d...,"Libération de Verviers par les alliés, 9/1944....",1764.0,verviers (belgique)--histoire--1944-1945 (libe...,Q31,Belgium,Q202954,Verviers,LOC_PLACE,Q56061,administrative territorial entity
5,00029795,LOC_BUILDING,hôtel de ville,http://pallas.cegesoma.be/pls/opac/opac.search...,Affiches de la libération (Harmonie badigeonné...,"Libération de Verviers par les alliés, 9/1944....",1764.0,verviers (belgique)--histoire--1944-1945 (libe...,Q31,Belgium,Q202954,Verviers,LOC_BUILDING,Q41176,building
6,00029795,LOC_PLACE,Verviers,http://pallas.cegesoma.be/pls/opac/opac.search...,Affiches de la libération (Harmonie badigeonné...,"Libération de Verviers par les alliés, 9/1944....",1764.0,verviers (belgique)--histoire--1944-1945 (libe...,Q31,Belgium,Q202954,Verviers,LOC_PLACE,Q56061,administrative territorial entity
7,00010703,LOC_PLACE,Malines,http://pallas.cegesoma.be/pls/opac/opac.search...,Enterrement tzigane au camp d'Elewijt près de ...,Funérailles d'un roi tsigane à Elewijt : 1940-...,716.0,guerre mondiale (1939-1945)--tsiganes--elewijt...,Q31,Belgium,Q1990353,Elewijt,LOC_PLACE,Q56061,administrative territorial entity


In [24]:
#gb["spatial_value"].get_group("00040082") # attention : changer l'id lorsqu'on resample

D:\Anaconda3\lib\site-packages\_pytest\fixtures.py:844: DeprecationWarning: The `convert` argument is deprecated in favor of `converter`.  It will be removed after 2019/01.
  params = attr.ib(convert=attr.converters.optional(tuple))
D:\Anaconda3\lib\site-packages\_pytest\fixtures.py:846: DeprecationWarning: The `convert` argument is deprecated in favor of `converter`.  It will be removed after 2019/01.
  ids = attr.ib(default=None, convert=_ensure_immutable_ids)


KeyError: '00040082'

In [25]:
df = pd.DataFrame(gb['spatial_value'].apply(lambda x: "%s" % ' '.join(x))).reset_index()

In [26]:
df.head()

,beeldid,spatial_value
0,00005881,Bruges
1,00010703,Malines
2,00029746,Verviers Palais de Justice Verviers
3,00029795,hôtel de ville Verviers
4,00038225,"Lycée Commun , d' Ixelles"


In [27]:
df['spatial_value'].apply(lambda x: Fingerprinter(x).get_fingerprint_nonsorted())

0                        bruges
1                       malines
2    verviers palais de justice
3       hotel de ville verviers
4         lycee commun  ixelles
Name: spatial_value, dtype: object

In [43]:
#transformer les loc_place pays en loc_country
with open(r"C:\Users\ettor\Documents\GitHub\UGESCO\data\countries.csv", 'r', encoding="utf8") as f:
    countries = set([unidecode(country.strip().lower()) for country in f])
    
def replace_countries(row):
    if unidecode(row['spatial_value'].strip().lower()) in countries:
        return "LOC_COUNTRY"

pd.Series(gb['ner_class'])

0                             (00005881, [LOC_PLACE])
1                             (00010703, [LOC_PLACE])
2    (00029746, [LOC_PLACE, LOC_BUILDING, LOC_PLACE])
3               (00029795, [LOC_BUILDING, LOC_PLACE])
4                          (00038225, [LOC_BUILDING])
dtype: object

In [36]:
#et maintenant, appliquer la concatenation uniquement aux groupes qui ne contiennent pas plus de trois loc_place
pd.Series(gb['ner_class'])

0                             (00005881, [LOC_PLACE])
1                             (00010703, [LOC_PLACE])
2    (00029746, [LOC_PLACE, LOC_BUILDING, LOC_PLACE])
3               (00029795, [LOC_BUILDING, LOC_PLACE])
4                          (00038225, [LOC_BUILDING])
dtype: object

In [ ]:
#appliquer get_nominatim